In [17]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import datetime 

In [4]:
#from matplotlib import pyplot as plt

In [24]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.impute import SimpleImputer

# 0. pozyskanie danych

In [35]:
movies =[]
url=['https://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2018&p=.htm',
     'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2018&p=.htm'
    ]

for i in range(2): 
    page = requests.get(url[i])
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    tr_elements = tr_elements[9:109]
    mov = []
    for xd in range(len(tr_elements)):
        for a in tr_elements[xd]:      
            try:
                mov.append(int(a.text_content()))
            except:
                mov.append(a.text_content())
                pass
            try:
                url_mov = a.getchildren()[0].getchildren()[0].getchildren()[0].attrib['href'] #XD
                page2 = requests.get('https://www.boxofficemojo.com'+url_mov)
                doc2 = lh.fromstring(page2.content)
                tr_elements2 = doc2.xpath('//tr')
                mov.append(tr_elements2[7].getchildren()[0].getchildren()[0].text_content()) #distributor full name
                mov.append(tr_elements2[7].getchildren()[1].getchildren()[0].text_content()) #release date
                mov.append(tr_elements2[8].getchildren()[0].getchildren()[0].text_content()) # genre
                mov.append(tr_elements2[8].getchildren()[1].getchildren()[0].text_content()) #runtime
                mov.append(tr_elements2[9].getchildren()[0].getchildren()[0].text_content()) #rating
                mov.append(tr_elements2[9].getchildren()[1].getchildren()[0].text_content()) #budget
                if(tr_elements2[15].getchildren()[0].text_content().replace(u'\xa0', u'').replace('+','') == 'Foreign:'):
                    mov.append(tr_elements2[15].getchildren()[1].text_content().replace(u'\xa0', u'')) #rest world
                else:
                    mov.append('no dist')
            except: 
                pass #XD2
        movies.append(mov)
        mov = []

In [36]:
headers =['#','name','distributor_full_name','release_date','genre','runtime','rating','budget','rest_world','studio','total_gross','theaters_total','opening','theaters_opening','open_date','close_date',]
df = pd.DataFrame(movies,columns = headers)
df.to_csv('movies.csv')

# 1. czyszczenie + imdb, cinemascore

In [18]:
df = pd.read_csv('movies.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date
0,0,0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,2 hrs. 20 min.,PG-13,NaN,"$646,853,595",BV,"$700,059,566","4,084","$202,003,951","4,020",2/16,8/9
1,1,1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,2 hrs. 36 min.,PG-13,NaN,"$1,369,544,272",BV,"$678,815,482","4,474","$257,698,183","4,474",4/27,9/13
2,2,2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,1 hrs. 58 min.,PG,NaN,"$634,223,615",BV,"$608,581,744","4,410","$182,687,905","4,410",6/15,12/13
3,3,3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,2 hrs. 8 min.,PG-13,$170 million,"$891,764,701",Uni.,"$417,719,760","4,485","$148,024,610","4,475",6/22,10/4
4,4,4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,2 hrs. 19 min.,PG-13,NaN,"$812,700,000",WB,"$335,061,807","4,184","$67,873,522","4,125",12/21,4/4


In [1]:
#df['total_gross'].isnull().values.any()
# #usuwanie $ itp
# df['total_gross'].apply(lambda x: str(x))
# df['opening'].apply(lambda x: str(x))
# df['total_gross'] = df['total_gross'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
# df['opening'] = df['opening'].replace('\.','', regex=True).replace(',','', regex=True).str[1:].astype(float)
# #df['opening'].fillna(df['opening'].mean(),inplace=True)
# df = df.sample(frac=1)
# # 'tasowanie' wierszy
#df.to_csv('movies.csv')
# df.head()

In [8]:
def str2int_w_na(col):
    df[col] = df[col].fillna(-1)
    df[col] = df[col].astype(int)
    df[col] = df[col].astype(str)
    df[col] = df[col].replace('-1', np.nan)

In [91]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

# len(df['studio'].unique()) #57
# len(df['genre'].unique()) #34
# len(df['rating'].unique()) #5

#runtime to int
df['runtime'] = df['runtime'].replace('hrs.','', regex=True).replace('min.','', regex=True).apply(lambda x: int(x.split()[0])*60 + int(x.split()[1]))

#budget to float (in milions)
df['budget'] = df['budget'].replace(' ','', regex=True).replace('million','', regex=True).str[1:]
df.loc[132,'budget'] =1
df['budget'] = df['budget'].astype(float)

#theters to int
df['theaters_total'] = df['theaters_total'].replace(',','', regex=True).astype(int)
df['theaters_opening'] = df['theaters_total'].replace(',','', regex=True).astype(int)

#rest_wrorld to int
df['rest_world'] = df['rest_world'].replace('no dist',float('nan'), regex=True)
df['rest_world'] = df['rest_world'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
str2int_w_na('rest_world')

#total_gross, opening to int
df['total_gross'] = df['total_gross'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
df['total_gross'] = df['total_gross'].astype(int)

df['opening'] = df['opening'].replace('\.','', regex=True).replace(',','', regex=True).str[1:]
str2int_w_na('opening')

#days in theaters
for x in range(len(df)):
    try:
        d1 = datetime.date(2018, int(df['open_date'][x].split("/")[0]), int(df['open_date'][x].split("/")[1]) )
        d2 = datetime.date(2018, int(df['close_date'][x].split("/")[0]), int(df['close_date'][x].split("/")[1]) )
        days = d2 - d1
        if days<datetime.timedelta(0):
            df.loc[x,'days_in_theaters']= (datetime.timedelta(365) + (d2 - d1)).days
        else:
            df.loc[x,'days_in_theaters']= (d2 - d1).days
    except:
        pass #przepraszam ;_;

#df.head()


KeyError: "['Unnamed: 0' 'Unnamed: 0.1'] not found in axis"

In [90]:
(datetime.timedelta(365) + (d2 - d1)).days

86

In [ ]:
#imdb oceny
from imdb import IMDb

ia = IMDb()

In [ ]:
import time

start = time.time()

for id in range(len(df)):
    try:
        movies = ia.search_movie(df.loc[id,'name'])
        movie = ia.get_movie(movies[0].movieID)
        df.loc[id,'imdb_rating']= movie['rating']
        directors=[]
        for person in movie['director']:    
            directors.append(person['name'])
        df.loc[id,'directors'] = ' & '.join(directors)
    except:
        df.loc[id,'directors'] = '???'
        
end = time.time()
print(end - start)
# 14 minut !!!

df.head()

In [ ]:
#df[['name','directors','imdb_rating']]
#len(df['directors'].unique()) #190

In [ ]:
#cinemascore
import cinemascore
import time

start = time.time()

for id in range(len(df)):
    try:
        x = cinemascore.search(df.loc[id,'name'])
        df.loc[id,'cinemascore_name'] = list(x)[0]
        df.loc[id,'cinemascore_rating'] = x[list(x)[0]]
    except:
        df.loc[id,'cinemascore_rating'] = np.nan
        pass
    
end = time.time()
print(str((end - start)/60)+" minut") # aprox. 5 min

In [21]:
df.head()

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,total_gross,theaters_total,opening,theaters_opening,open_date,close_date,days_in_theaters
0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140,PG-13,NaN,646853595,BV,700059566,4084,202003951,4084,2/16,8/9,174 days
1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156,PG-13,NaN,1369544272,BV,678815482,4474,257698183,4474,4/27,9/13,139 days
2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118,PG,NaN,634223615,BV,608581744,4410,182687905,4410,6/15,12/13,181 days
3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128,PG-13,170.0,891764701,Uni.,417719760,4485,148024610,4485,6/22,10/4,104 days
4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139,PG-13,NaN,812700000,WB,335061807,4184,67873522,4184,12/21,4/4,104 days


In [95]:
df.to_csv('movies_clean.csv',index=False)

In [ ]:
#df['budget'].unique()
#df['budget'].astype(float)
#df[df['budget']=='$170 million']
#df['budget'].isnull().sum() #104
#f[df['budget'].notnull()]
#df['budget'] = df['budget'].replace(' ','', regex=True).replace('million','', regex=True).str[1:]
#type(df.loc[36,'rest_world'])
#type(df['total_gross'][0])
#type(df['rest_world'][0])

In [ ]:
# critics reviews

# -key actors
# -director
# -season of year
# -other movies in cinema at that time
# -„hype index”
# china 

# 2. model

In [96]:
df = pd.read_csv('movies_clean.csv')
df.head()

,#,name,distributor_full_name,release_date,genre,runtime,rating,budget,rest_world,studio,...,theaters_total,opening,theaters_opening,open_date,close_date,days_in_theaters,imdb_rating,directors,cinemascore_rating,cinemascore_name
0,1,Black Panther,Buena Vista,"February 16, 2018",Action / Adventure,140,PG-13,NaN,6.468536e+08,BV,...,4084,202003951.0,4084,2/16,8/9,174.0,7.3,Ryan Coogler,A+,BLACK PANTHER (2018)
1,2,Avengers: Infinity War,Buena Vista,"April 27, 2018",Action / Adventure,156,PG-13,NaN,1.369544e+09,BV,...,4474,257698183.0,4474,4/27,9/13,139.0,8.5,Anthony Russo & Joe Russo,A,AVENGERS: INFINITY WAR (2018)
2,3,Incredibles 2,Buena Vista,"June 15, 2018",Animation,118,PG,NaN,6.342236e+08,BV,...,4410,182687905.0,4410,6/15,12/13,181.0,7.7,Brad Bird,A+,"INCREDIBLES 2, THE (2018)"
3,4,Jurassic World: Fallen Kingdom,Universal,"June 22, 2018",Action / Adventure,128,PG-13,170.0,8.917647e+08,Uni.,...,4485,148024610.0,4485,6/22,10/4,104.0,6.2,J.A. Bayona,A-,JURASSIC WORLD: FALLEN KINGDOM (2018)
4,5,Aquaman,Warner Bros.,"December 21, 2018",Action / Adventure,139,PG-13,NaN,8.127000e+08,WB,...,4184,67873522.0,4184,12/21,4/4,104.0,7.1,James Wan,A-,AQUAMAN (2018)


In [59]:
# del df['budget']
# df.columns

Index(['#', 'name', 'distributor_full_name', 'release_date', 'genre',
       'runtime', 'rating', 'rest_world', 'studio', 'total_gross',
       'theaters_total', 'opening', 'theaters_opening', 'open_date',
       'close_date', 'days_in_theaters', 'imdb_rating', 'directors',
       'cinemascore_rating', 'cinemascore_name'],
      dtype='object')

In [98]:
#brakujace wartosci - wykres
#df.isnull().sum().to_frame(name='count').sort_values(by='count', ascending=False)['count'].plot(kind='bar', figsize=(20,10))

dfna=df.dropna(axis=0)
len(dfna) #55

55

In [129]:
# feats = ['opening']
# X = df[ feats ].values
# y = df['total_gross'].values

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3) 

# X_train.shape, X_test.shape
#
#df['opening'].notnull().sum() <------- 2
#df['opening'].fillna(df['opening'].mean(),inplace=True) # nie powinieniem tak robic wiem ale to tylko do testu, bo dla dwoch wiereszy jest NA
#

target = 'total_gross'
#features = [c for c in dfna.columns if c != 'total_gross']
features = ['runtime', 'rest_world','theaters_total', 'opening', 'theaters_opening','days_in_theaters', 'imdb_rating']

train,test = train_test_split(dfna, test_size=0.33)

model = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
model.fit(train[features],train[target])
print(model.score(test[features],test[target]))

0.8936429740635896
